In [168]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, pickle

In [169]:
driver = webdriver.Chrome()

In [170]:
links_to_follow = []
first_names = {}
previous_link = None

In [171]:
offset = 0
while offset < 400:
    driver.get("https://dataspace.princeton.edu/jspui/handle/88435/dsp01d504rk39g?offset={}".format(offset))
    driver.refresh()
    time.sleep(2)
    all_links = driver.find_elements_by_tag_name("a") # get links
    for link in all_links:
        current_link = link.get_attribute("href")
        if "author&value" in current_link:
            links_to_follow.append(previous_link)
            split_link = current_link.split("+")
            first_names[previous_link] = split_link[-1]
        previous_link = current_link
    offset += 20

In [172]:
for link in first_names:
    if "&" in first_names[link]:
        index = first_names[link].index("&")
        first_names[link] = first_names[link][:index]
    first_names[link] = first_names[link].rstrip(".")

In [173]:
all_pairs = []
for link in first_names:
    if "author" not in link:
        driver.get(link)
        driver.refresh()
        time.sleep(1)
        try:
            element = driver.find_element_by_class_name("dc-description-abstract")
            if element != None:
                all_pairs.append((first_names[link], element.text))
                print(len(all_pairs))
        except:
            print("no abstract found")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
no abstract found
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
no abstract found
no abstract found
no abstract found
no abstract found
no abstract found
no abstract found
no abstract found
no abstract found
no abstract found
no abstract found
no abstract found
no abstract found
no abstract found
no abstr

KeyboardInterrupt: 

In [174]:
all_names = {}
with open("girls_names") as f:
    for line in f.readlines():
        split_line = line.split()
        all_names[split_line[0]] = [int(split_line[1]), 0]
with open("boys_names") as f:
    for line in f.readlines():
        split_line = line.split()
        if split_line[0] in all_names:
            all_names[split_line[0]][1] = int(split_line[1])
        else:
            all_names[split_line[0]] = [0, int(split_line[1])]
for name in all_names:
    frequency = all_names[name][0] + all_names[name][1]
    all_names[name][0] /= frequency
    all_names[name][1] /= frequency

In [175]:
personal_words = ["my", "i"]
collective_words = ["our", "we"]

In [176]:
female = {"personal": 0, "collective": 0, "ambiguous": 0}
male = {"personal": 0, "collective": 0, "ambiguous": 0}
for pair in all_pairs:
    name = pair[0]
    if name not in all_names:
        continue
    if all_names[name][0] > 0.1 and all_names[name][0] < 0.9:
        continue
    female_status = True if all_names[name][0] > 0.5 else False
    
    abstract = pair[1]
    personal_count = 0
    collective_count = 0
    for word in abstract.split():
        stripped_word = word.lower().rstrip(",:.;")
        if stripped_word in personal_words:
            personal_count += 1
        elif stripped_word in collective_words:
            collective_count += 1
    
    personal = True if (personal_count > 0 and collective_count == 0) else False
    collective = True if (personal_count == 0 and collective_count > 0) else False
    ambiguous = True if (not personal_count and not collective_count) else False
    
    if female_status:
        if personal:
            female["personal"] += 1
        elif collective:
            female["collective"] += 1
        else:
            female["ambiguous"] += 1
    else:
        if personal:
            male["personal"] += 1
        elif collective:
            male["collective"] += 1
        else:
            male["ambiguous"] += 1

In [177]:
print("Female")
female_sum = female["personal"] + female["collective"] + female["ambiguous"]
print("\tPersonal:", female["personal"]/female_sum)
print("\tCollective:", female["collective"]/female_sum)
print("\tAmbiguous:", female["ambiguous"]/female_sum)
print("Total:", female_sum)

Female
	Personal: 0.06557377049180328
	Collective: 0.36065573770491804
	Ambiguous: 0.5737704918032787
Total: 61


In [178]:
print("Male")
male_sum = male["personal"] + male["collective"] + male["ambiguous"]
print("\tPersonal:", male["personal"]/male_sum)
print("\tCollective:", male["collective"]/male_sum)
print("\tAmbiguous:", male["ambiguous"]/male_sum)
print("Total:", male_sum)

Male
	Personal: 0.08333333333333333
	Collective: 0.38333333333333336
	Ambiguous: 0.5333333333333333
Total: 60


In [179]:
pickle.dump(all_pairs, open("cbe_pairs.p", "wb")) # note: messed up the ORF ones - overrode w POL